In [25]:
import pandas as pd
import time
import os

In [ ]:
# 定义行业范围
sectors = range(10, 65, 5)

# 设置数据目录路径 - 硬编码为my_outputs
DATA_DIR = "./data_processor/my_outputs"
FUNDAMENTAL_FILE = f"{DATA_DIR}/final_ratios.csv"

print(f"使用数据目录: {DATA_DIR}")
print(f"基本面数据文件: {FUNDAMENTAL_FILE}")
print(f"行业范围: {list(sectors)}")

In [27]:
#gvkey is unique identifier
df_dict = {'gvkey':[], 'predicted_return':[], 'trade_date':[]}

In [ ]:
# ===== 新代码：运行my_outputs目录下所有行业的股票选择 =====
start=time.time()
print("开始运行所有行业的股票选择模型...")

for sector in sectors:
    sector_file = f"{DATA_DIR}/sector{sector}.xlsx"
    print(f"\n正在处理 sector{sector}...")
    
    # 运行模型训练 - 使用my_outputs目录的文件
    cmd = f"python fundamental_run_model.py -sector_name sector{sector} -tic_column gvkey -fundamental {FUNDAMENTAL_FILE} -sector {sector_file}"
    print(f"执行命令: {cmd}")
    
    os.system(cmd)
    
    # 读取预测结果
    df = pd.read_csv(f"results/sector{sector}/df_predict_best.csv", index_col=0)
    print(f"  读取预测结果: {df.shape[0]} 个日期, {df.shape[1]} 只股票")
    
    for idx in df.index:
        predicted_return = df.loc[idx]
        top_q = predicted_return.quantile(0.75)
        predicted_return = predicted_return[predicted_return >= top_q]
        for gvkey in predicted_return.index:
            df_dict["gvkey"].append(gvkey)
            df_dict["predicted_return"].append(predicted_return[gvkey])
            df_dict["trade_date"].append(idx)
    
    print(f"  ✓ sector{sector} 处理完成")

end=time.time()

# ===== 原有代码（已注释） =====
# start=time.time()
# for sector in sectors:
#     os.system(f"python3 fundamental_run_model.py -sector_name sector{sector} -tic_column gvkey -fundamental final_ratios.csv -sector sector{sector}.xlsx ")
#     df = pd.read_csv(f"results/sector{sector}/df_predict_best.csv", index_col=0)
#     for idx in df.index:
#         predicted_return = df.loc[idx]
#         top_q = predicted_return.quantile(0.75)
#         predicted_return = predicted_return[predicted_return >= top_q]
#         for gvkey in predicted_return.index:
#             df_dict["gvkey"].append(gvkey)
#             df_dict["predicted_return"].append(predicted_return[gvkey])
#             df_dict["trade_date"].append(idx)
# end=time.time()

In [6]:
print("总耗时: ", (end-start)/60, ' 分钟')
print(f"处理完成！总记录数: {len(df_dict['gvkey'])}")

# 原有代码（已注释）
# print("it took ", (end-start)/60, ' minutes')

总耗时: 0.40141645272572835 分钟
处理完成！总记录数: 0


In [6]:
df_result = pd.DataFrame(df_dict)

In [7]:
# 保存结果到CSV文件 - 使用新的文件名
output_file = "stock_selected_my_outputs.csv"
df_result.to_csv(output_file, index=False)
print(f"结果已保存到: {output_file}")

# 原有代码（已注释）
# df_result.to_csv("stock_selected.csv")

结果已保存到: stock_selected_my_outputs.csv


In [9]:
df_result.head()

,gvkey,predicted_return,trade_date
